In [12]:
%load_ext autoreload
%autoreload 2

import warnings
import pandas as pd
import numpy as np
import os
import sys # error msg, add the modules
import operator # sorting
from math import *
import matplotlib.pyplot as plt

sys.path.append('../')

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

import prep
from warp import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
target_kern_name = 'mcx_main_loop'
trace_file = 'trace_1e6.csv'
metrics_file = 'metrics_1e6.csv'
sass_result = 'mcx.sm52.sass.result'
    
# read trace and metrics into dataframe, from generated trace file
df_kernel_trace, df_metrics = prep.Prep_trace_metrics(trace_file, metrics_file)
    
out_columns = prep.init_df_columns(df_metrics)
df_kern = pd.DataFrame(columns=out_columns)
    
# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
    
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

In [14]:
df_kern

,kern_name,duration_ms,gridx,gridy,gridz,blkx,blky,blkz,reg_per_thread,shared_mem,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,mcx_main_loop,181.664061,192.0,1.0,1.0,64.0,1.0,1.0,68.0,2250.0,...,Low (3),23.253262GB/s,242580238,Low (1),0.000000B/s,Low (1),Low (1),5.086808,3.642482%,0.000000%


In [15]:
df_kern.to_csv('kernel_stat.csv', index=False, encoding='utf-8')

In [16]:
warp_inst = WarpInst()
warp_inst.run(df_kern, sass_result)

ldg (global load) clks (per warp) 	 = 10632700.0
stg (global store) clks (per warp) 	 = 57.0
lds (shared load) clks (per warp) 	 =  404378.0
sts (shared store) clks (per warp) 	 = 119225.0
=> Memory inst. (per warp) 		 = 11156360.0 (clocks)

Integer inst. (per warp) 	= 17139240.0 (clocks)
FP32 inst. (per warp) 		= 21513795.0 (clocks)
FP64 inst. (per warp) 		= 955011.0 (clocks)
Compute ldst inst. (per warp) 	= 2482380.0 (clocks)
=> Compute inst. (per warp) 	= 42090426.0 (clocks)

Compute-intensive Warp : compute_clk / mem_clk = 3.77277409478
